In [ ]:
import torch
import torchvision
from torchvision import transforms,datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
train = torchvision.datasets.MNIST('',train=True,download=True,transform=transforms.Compose([transforms.ToTensor()]))
test = torchvision.datasets.MNIST('',train=False,download=True,transform=transforms.Compose([transforms.ToTensor()]))

trainset=torch.utils.data.DataLoader(train,batch_size=10,shuffle=True, num_workers=4)
testset=torch.utils.data.DataLoader(test,batch_size=10,shuffle=True, num_workers=4)

In [ ]:
trainset_shape=trainset.dataset.train_data.shape
testset_shape=testset.dataset.test_data.shape
print(trainset_shape,testset_shape)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(784,400)
        self.linear2 = nn.Linear(400,250)
        self.linear3 = nn.Linear(250,100)
        self.linear4 = nn.Linear(100,10)
        
    def forward(self,X):
        X = F.relu(self.linear1(X))
        X = F.relu(self.linear2(X))
        X = F.relu(self.linear3(X))
        X = self.linear4(X)
        return F.log_softmax(X, dim=1)

net=Net()
print(net)

In [ ]:

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(),lr=0.001)

In [ ]:
%%time
loss_arr = []
loss_epoch_arr = []
max_epochs = 4
for epoch in range(max_epochs):
    #iterate through all the batches in each epoch
    for i, data in enumerate(trainset, 0):
        #keeping the network in training mode     
        net.train()     
        inputs, labels = data 
        #clear the gradients     
        net.zero_grad()     
        #forward pass     
        net(inputs.view(-1,784))
        outputs = net(inputs)      
        loss = loss_fn(outputs, labels)     
        #backward pass     
        loss.backward()     
        net.step()     
        loss_arr.append(loss.item()) 
    loss_epoch_arr.append(loss.item()) 

In [ ]:
# function to do evaluation (calculate the accuracy) in gpu
def evaluation(dataset):
    total, correct = 0, 0
    #keeping the network in evaluation mode 
    net.eval()
    for data in dataset:
        inputs, labels = data
        outputs = net(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total